In [2]:
import datetime

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn


%matplotlib inline

In [3]:
%%time
dt = pd.read_csv('log.csv', parse_dates=['event_date'])
dt['event_date'] = dt['event_date'].astype(datetime.datetime)

CPU times: user 56.2 s, sys: 1.41 s, total: 57.7 s
Wall time: 57.6 s


In [4]:
pd.options.display.max_rows = 100

_____

In [5]:
dt['week_days'] = np.array(list(map( lambda x: x.isoweekday(),
                           dt.event_date.values))).astype(int)

dt['rel_time'] = np.array(list(map(lambda dt: (dt.minute*60 + dt.second), 
                                    dt.event_date.values))).astype(float)

In [6]:
dt.head(3)

,log_id,event_date,eventtype_id,eventtype_group,eventype_name,is_event_pageview,ClientSideAppName,cookie_id,cookie,user_id,category_id,ParentCategoryName,CategoryName,location_id,useragent_id,useragent,ip,week_days,rel_time
0,1090142709087,2016-10-21 00:00:03,57,Request_ItemView,Item View,True,Android App,69412,7cce9961-ad09-41d2-8199-e333398*****,NaN,22.0,Личные вещи,"Одежда, обувь, аксессуары",3953.0,2.566710e+05,Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US...,82.138.25.1,5,3.0
1,1090142707471,2016-10-21 00:00:06,57,Request_ItemView,Item View,True,iPhone App,13845,6419167A-809E-498F-B8A7-C43CE0D*****,NaN,34.0,Транспорт,Запчасти и аксессуары,1261.0,2.366423e+11,"AVITO 7.0, (iPhone; 9.3.3; ru_RU)",2.93.60.1,5,6.0
2,1090150202477,2016-10-21 00:00:07,57,Request_ItemView,Item View,True,Android App,69472,836d5d7c-3e3e-4006-91ff-50dbd54*****,NaN,16.0,Хобби и отдых,Музыкальные инструменты,346.0,2.566710e+05,Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US...,82.138.25.1,5,7.0


In [7]:
pd.options.display.max_columns=100

In [ ]:
plt.figure(figsize=(15,8))
plt.hist(dt[dt.location_id == 3953.0].event_date.values, bins = 1000);

In [44]:
dt[dt.cookie.str.len() < 20].is_bad_ip.value_counts()

False    7663905
True      162882
Name: is_bad_ip, dtype: int64

In [45]:
dt[dt.cookie.str.len() > 20].is_bad_ip.value_counts()

False    3387362
True      393651
Name: is_bad_ip, dtype: int64

In [63]:
dt[dt.is_bad_ip].eventtype_group.value_counts() / dt[~dt.is_bad_ip].eventtype_group.value_counts() 

 Event_Item_CallPhone            0.021659
Request                          0.015444
Request_FavoritesItems           0.017244
Request_ItemAdd                  0.016315
Request_ItemClose                0.021066
Request_ItemEdit                 0.014473
Request_ItemMessage              0.019742
Request_ItemSearch               0.018740
Request_ItemView                 0.088458
Request_MainView                 0.014512
Request_ShopsSearch              0.029158
Request_UserAccountVAS           0.019110
Request_UserEditContacts         0.022989
Request_UserEditNotifications    0.040816
Request_UserItemSearch           0.014477
Request_UserLogin                0.022300
Request_UserRegistration         0.017431
Name: eventtype_group, dtype: float64

In [47]:
dt[dt.is_bad_ip].cookie.str.len().value_counts()

36    393586
16    154291
15      8018
14       569
32        62
13         4
31         3
Name: cookie, dtype: int64

In [48]:
dt[~dt.is_bad_ip].cookie.str.len().value_counts()

16    6700909
36    3381021
15     936407
14      25938
30       6333
13        629
12         22
32          8
Name: cookie, dtype: int64

# по времени гистограмма 1 запросов 

In [13]:
tmp_count = dt.groupby('cookie_id').eventtype_id.count().reset_index()
tmp_count.columns = ['cookie_id', 'cookie_counts']

In [14]:
dt['is_avito'] = dt.useragent.str.lower().str.contains('avito')

dt['is_win'] = dt.useragent.str.lower().str.contains('indow')
dt['how_many_nans'] = dt.category_id.isnull().astype(int) + dt.user_id.isnull().astype(int) 

In [15]:
dt.shape

(11607800, 22)

In [16]:
dt.category_id = dt.category_id.fillna(-10)
dt.location_id = dt.location_id.fillna(-199.)

In [17]:
%%time

tmp_count = dt.groupby('cookie_id').eventtype_id.count().reset_index()
tmp_count.columns = ['cookie_id', 'cookie_counts']

tmp_count1 = dt.category_id.groupby(dt.cookie_id).apply(lambda x: len(x.unique())).reset_index()
tmp_count1.columns = ['cookie_id', 'category_id_counts']

tmp_count2 = dt.eventtype_id.groupby(dt.cookie_id).apply(lambda x: len(x.unique())).reset_index()
tmp_count2.columns = ['cookie_id', 'eventtype_id_unique']

tmp_count = pd.merge(tmp_count1, tmp_count)
tmp_count = pd.merge(tmp_count2, tmp_count)

CPU times: user 42.9 s, sys: 456 ms, total: 43.4 s
Wall time: 42.8 s


In [18]:
tmp_count.shape

(438657, 4)

In [19]:
%%time

tmp_count_is_10 = dt.eventtype_id.groupby(dt.cookie_id).apply(lambda x: (10 in x.unique())).reset_index()
tmp_count_is_10.columns = ['cookie_id', 'is_10_in_eventtype_id']
tmp_count = pd.merge(tmp_count_is_10, tmp_count)
tmp_count.is_10_in_eventtype_id = tmp_count.is_10_in_eventtype_id.apply(int)

CPU times: user 23.6 s, sys: 272 ms, total: 23.9 s
Wall time: 23.3 s


In [20]:
tmp_count.shape

(438657, 5)

In [21]:
%%time

tmp_count_nulls = dt.how_many_nans.groupby(dt.cookie_id).apply(lambda x: (x.mean())).reset_index()
tmp_count_nulls.columns = ['cookie_id', 'nans_mean']
tmp_count = pd.merge(tmp_count_nulls, tmp_count)


CPU times: user 44.4 s, sys: 164 ms, total: 44.5 s
Wall time: 44.5 s


In [22]:
tmp_count.shape

(438657, 6)

In [23]:
%%time

ip_uniq_num = dt.ip.groupby(dt.cookie_id).apply(lambda x: len(x.unique())).reset_index()
ip_uniq_num.columns = ['cookie_id', 'ip_uniq_counts']
tmp_count = pd.merge(ip_uniq_num, tmp_count)

CPU times: user 24.7 s, sys: 0 ns, total: 24.7 s
Wall time: 24.7 s


In [24]:
tmp_count.shape

(438657, 7)

In [25]:
%%time

location_id_uniq_num = dt.location_id.groupby(dt.cookie_id).apply(lambda x: len(x.unique())).reset_index()
location_id_uniq_num.columns = ['cookie_id', 'location_id_uniq_counts']
tmp_count = pd.merge(location_id_uniq_num, tmp_count)


CPU times: user 20.8 s, sys: 52 ms, total: 20.9 s
Wall time: 20.8 s


In [26]:
tmp_count['uniq_location_ratio'] = tmp_count['location_id_uniq_counts'] * 1.0 / tmp_count['cookie_counts']

tmp_count['uniq_ip_ratio'] = tmp_count['ip_uniq_counts'] * 1.0 / tmp_count['cookie_counts']

In [27]:
%%time 

tmp_time = dt.groupby('cookie_id')['rel_time'].std().reset_index()
tmp_time = tmp_time.fillna(-1)
grouped_df = pd.merge(tmp_time, tmp_count)

CPU times: user 388 ms, sys: 0 ns, total: 388 ms
Wall time: 387 ms


In [28]:
grouped_df = pd.merge(tmp_time, tmp_count)

In [29]:
dt.is_win = dt.is_win.fillna(False)
dt.is_win = dt.is_win.astype(int)
dt.is_avito = dt.is_avito.fillna(False)
dt.is_avito = dt.is_avito.astype(int)
tmp_count =  dt.is_win.groupby(dt.cookie_id).apply(lambda x: np.max(x)).reset_index()
tmp_count1 =  dt.is_avito.groupby(dt.cookie_id).apply(lambda x: np.max(x)).reset_index()

tmp_count.head()
grouped_df = pd.merge(tmp_count1, pd.merge(tmp_count, grouped_df))

In [30]:
assert grouped_df.shape[0] == 438656+1

In [31]:
bad_ip = set(dt.ip.groupby(dt.cookie_id).first()[grouped_df.cookie_counts == 1])
ips = dt.ip.unique()

d = {}
for i in ips:
    d[i] = i in bad_ip

res = dt.ip.map(d)
dt['is_bad_ip'] = res

In [32]:
dt[dt.cookie_id==4].head()

,log_id,event_date,eventtype_id,eventtype_group,eventype_name,is_event_pageview,ClientSideAppName,cookie_id,cookie,user_id,category_id,ParentCategoryName,CategoryName,location_id,useragent_id,useragent,ip,week_days,rel_time,is_avito,is_win,how_many_nans,is_bad_ip
11184378,1117952486545,2016-10-31 09:30:22,3,Request_UserLogin,User Login,True,Android App,4,38acc04ab19*****,4.648250e+10,-10.0,NaN,NaN,-199.0,1.859365e+11,AVITO 3.9 (GT-I8160; Android 2.3.6; ru),79.140.25.53,1,1822.0,1,0,1,True


In [33]:
assert dt.shape[0] == 11607800

In [34]:
tmp_bad_ip = dt.groupby('cookie_id')['is_bad_ip'].apply(lambda x: np.sum(x)!=0).reset_index()
next_grouped_df = pd.merge(tmp_bad_ip, grouped_df)

In [35]:
next_grouped_df.shape

(438657, 14)

In [36]:
a = dt.groupby('cookie_id').user_id.apply(lambda x: np.isnan(x).astype(int)).reset_index()
dt['isnaninid'] = a.user_id
tmp_count = dt.groupby('cookie_id').isnaninid.sum().reset_index()

In [37]:
next_grouped_df = pd.merge(tmp_count, next_grouped_df)

In [38]:
tmp_loc_count = dt.groupby('cookie_id').location_id.count().reset_index()
tmp_loc_count.columns = ['cookie_id', 'loc_count']

next_grouped_df = pd.merge(tmp_loc_count, next_grouped_df)

In [39]:
useragent_count1 = dt.useragent.groupby(dt.cookie_id).count().reset_index()
useragent_count1.columns = ['cookie_id', 'useragent_count']

In [40]:
next_grouped_df = pd.merge(useragent_count1, next_grouped_df)

_____

In [ ]:
(next_grouped_df.is_bad_ip==False) &
(next_grouped_df.is_win==0) &
(next_grouped_df.is_avito==1) &
(next_grouped_df.rel_time != 0.) & 
(next_grouped_df.rel_time != 1.) &
(next_grouped_df.uniq_ip_ratio != 1.) &
(next_grouped_df.uniq_location_ratio != 1.) 

In [49]:
ans = next_grouped_df[['cookie_id']].copy()
ans['IsHuman'] = False
assert ans.shape[0] == 438656+1

In [50]:
ans.loc[ 
        (next_grouped_df.is_bad_ip==False) &
        #(next_grouped_df.is_win==0) & 
        (next_grouped_df.is_avito==1) &
        (next_grouped_df.rel_time != 0.)   
#         (next_grouped_df.rel_time != 1.) &
        & ((next_grouped_df.uniq_ip_ratio != 1.) | (next_grouped_df.cookie_counts <= 6.))
#         (next_grouped_df.uniq_location_ratio != 1.)  
        & (next_grouped_df.useragent_count > 1)
        & (0 != next_grouped_df.useragent_count)
        # сильное заявление что людей с виндой нету
        # что нету людей среди тех у кого одна кука (проверено одним сабмитом)
        # и что нету в плохих айпишках
        ,
        'IsHuman'] = True

ans.IsHuman.sum() / ans.IsHuman.count(), 1-ans.IsHuman.sum() / ans.IsHuman.count()

(0.10603273172433131, 0.89396726827566875)

In [41]:
0.9027

0.9027

In [285]:
ans.IsHuman = ans.IsHuman.astype(int)
assert ans.shape[0] == 438656+1

In [288]:
ans.sort_values('cookie_id').to_csv('amir_Last90_9.02_subm.csv', index=False)

In [289]:
ans.IsHuman.sum(), ans.IsHuman.count()

(46512, 438657)

In [290]:
print ('run!?????????')

run!?????????


In [ ]:
0.8923


x + y = 0.10554031965750005,
z + m = 0.89245811647825068

z + x = 0.10754188352174934 (меняется)

(2-2/0.8923)x + y + z


In [132]:
x1 =   711898241257741 / 7488413958568295 
x2 =   7843136165724 / 7488413958568295 
x3 =   93419900437252 / 7488413958568295 
x4 =   4748361920280596 / 5395970930144671 

In [133]:
print (x1, x3)
print (x2, x4)

0.09506662494842211 0.012475258573327173
0.0010473694709077653 0.8799828579049235


In [ ]:
### досчитатть! пока самое лучшее попытка

In [109]:
(2-2/0.9092)

-0.2269235051775973

In [ ]:
x1 =   386413271128096 4052755386177969 
x2 =   50966425922556 4999402603231555 
x3 =   35864813299259 4052755386177969 
x4 =   5180382548329873 5840636047044759 
 

In [80]:
print (386413271128096 / 4052755386177969 , 35864813299259 / 4052755386177969 )
print (50966425922556 / 4999402603231555 , 5180382548329873 / 5840636047044759)

0.09534581644033312 0.00884948877535933
0.010194503217166768 0.8869552060089482


# чемпион

In [ ]:
0.9090
(next_grouped_df.cookie_counts >= 2) & 
(next_grouped_df.is_bad_ip==False) &
(next_grouped_df.is_win==0) & 
(next_grouped_df.is_avito==1) &
(next_grouped_df.rel_time != 0.)  & 
(next_grouped_df.rel_time != 1.) &
(next_grouped_df.uniq_ip_ratio != 1.) &
(next_grouped_df.uniq_location_ratio != 1.)

_______-

In [ ]:
f1 = 0.1901

In [ ]:
(2 - 2 * f1) /  (f1 + 2 - 2* f1)

In [223]:
1.-0.8949665727388253

0.1050334272611747